In [1]:
import copy
import pandas as pd
import numpy as np

import datetime as dt
import random
import time
import math
import itertools

#import statsmodels.api as sm

from IPython.display import clear_output


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from graphviz import Digraph
from scipy.ndimage import shift
#import pydot


# from k2_alg import k2 
# from k2_alg import gen_clean_df

from scikit_alarm_framework.alarm_generator import set_point

In [8]:
#instalacao sckit alarm
from getpass import getpass

import os

os.environ['GITHUB_AUTH'] = getpass('GitHub user') + ':' + getpass('GitHub password')
!pip -q install -e git+https://${GITHUB_AUTH}@github.com/abugim/scikit-alarm-framework.git@develop#egg=scikit-alarm-framework


GitHub user········
GitHub password········
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
# https://pypi.python.org/pypi/pydot
!pip install graphviz

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
!pip install --upgrade pip


^C
Operation cancelled by user
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
def get_lim_index(cdf, lim):
    summation = 0
    index = 0
    for i in np.arange(0, cdf.size):
        if cdf[i] > lim:
            index = i
            break
    return index 

In [11]:
def joint_probability(k,l, h, a, b):
    '''
        k B time horizon
        l A time horizon
        h instant in the future of serie B
        
        a, b array type'''

    #Alarm Series A (cause), B (effect), same len
    #tested
    sizeSeries = a.size
    transEntropy = 0
    numStates = 2**(k + l  + 1)
    combinations = list(map(list, itertools.product([0, 1], repeat=k+l+1)))
    counting = np.zeros(numStates)
    prob_cnjt = np.zeros(numStates)
    a_prob_ind = []
    b_prob_ind = []
    #joitn probability p(i_sub_t+1), i_sub_t**k, j_sub_t**l)
    inicio = np.max([k,l]) - 1
    for i in np.arange(inicio, sizeSeries - h):
        for hk in np.arange(0,k):
                b_prob_ind.append(b[i - hk])
        for hl in np.arange(0,l):
                a_prob_ind.append(a[i - hl])

        #print(a.size, b.size, a.size -1)     
        ab = [b[i + h]] + b_prob_ind + a_prob_ind 
        index_comb = combinations.index(ab)
        counting[index_comb] = counting[index_comb] + 1

        a_prob_ind = []
        b_prob_ind = []

    total = sum(counting)
  
    prob_cnjt = counting/total
     
    return prob_cnjt

In [12]:
#Joint probability evaluation p(i_t+h, i_t**k)
#tested
def joint_prob_ih_ik(k,l, joint_prob_ih_ik_jl):
    states_ith_ik = list(map(list, itertools.product([0, 1], repeat=k + 1)))
    combinations = list(map(list, itertools.product([0, 1], repeat=k+l+1))) 
    p_jnt_ith_ik = np.zeros(2**(k+1))
    
    for i, state in enumerate(states_ith_ik):
        for j, comb in enumerate(combinations):
            if comb[0:k+1] == state:
                p_jnt_ith_ik[i] = p_jnt_ith_ik[i] + joint_prob_ih_ik_jl[j]
    return p_jnt_ith_ik

In [14]:
#Division of the conditionals in log2 
#tested
def conditional_div(k,l,conditional_num, conditional_den):
    combinations = list(map(list, itertools.product([0, 1], repeat=k+l+1)))
    conditional_division = np.zeros(conditional_num.size)
    states_den = list(map(list, itertools.product([0, 1], repeat=1+k)))
    for j, comb in enumerate(combinations):
        if(conditional_den[states_den.index(comb[0:k+1])] != 0):
            conditional_division[j] = conditional_num[j]/conditional_den[states_den.index(comb[0:k+1])]            
    return conditional_division

In [18]:
def graph_simple(df, eng = 'dot'):
    edge_style = ""
    g = Digraph(engine=eng)
    in_graph = []
    for k, row in enumerate(df.index):
        if any(df.loc[row]):
            g.node(str(row),row, shape='oval', fontsize='10', width='0', style='filled', fillcolor='#c9c9c9', color="gray")
            in_graph.append(row)

              
    for c, col in enumerate(df.columns):
        if any(df[col]):
            if col not in in_graph:
                g.node(str(col), col, shape='oval', fontsize='10', width='0', style='filled', fillcolor='#c9c9c9', color="gray") 

    for j, col in enumerate(df.columns):
        for i, row in enumerate(df.index):
            if(df[col][i]):
                g.edge(str(row), str(col), label=str(df.at[row,col]), style= edge_style, color='black')  
    return g 

In [19]:
def graph(df, df_lag, eng = 'dot'):
    edge_style = ""
    g = Digraph(engine=eng)
   
    for k, row in enumerate(df.index):
        if any(df.iloc[k]) or any(df[row]):
            g.node(str(k),row, shape='oval', fontsize='10', width='0', style='filled', fillcolor='#c9c9c9', color="gray") 

    for j, col in enumerate(df.columns):
        for i, row in enumerate(df[col]):
            if(row):
                te_val  = str(np.round(row, 6))
                g.edge(str(i), str(j), label=str(df_lag[df_lag.columns[j]][i]),style= edge_style, color='dark')  
    return g 

In [20]:
def graph_from_dict(dictionary, eng = 'dot'):
    edge_style = ""
    g = Digraph(engine=eng)
   
    for k, i in dictionary.items():
        g.node(str(k),k, shape='oval', fontsize='10', width='0', style='filled', fillcolor='#c9c9c9', color="gray") 
        df_te.m
    for k, i in dictionary.items():
        for it in i:
            g.edge(str(it), str(k), label='',style= edge_style, color='dark')  
    return g 

In [21]:
def generate_df_valid_corrs(df, limit):
    df_valid = df.copy()
    for row in df.columns:
        for col in df.columns:
            if df[row][col] < limit:
                df_valid[row][col] = 0
                
    return df_valid 

In [273]:
##for paper test
def joint_probability_new(k, df, labels):
    '''
        k B time horizon
        l A time horizon
        h instant in the future of serie B
        
        
        a, b array type'''
    
    numStates=2**(np.sum(k))
    combinations = list(map(list, itertools.product([False, True], repeat=np.sum(k))))
    print(combinations)
    prob_cnjt = np.zeros(numStates)

    gpd = df.groupby(labels, as_index=False).size().reset_index(name='Count')
    display(gpd)
    display(combinations)
    total = sum(gpd['Count'])
    
    for i in np.arange(0,gpd.shape[0]):
        comb = [e for e in gpd.iloc[i][0:len(k)].values.tolist()]
        idx = combinations.index(comb)
        prob_cnjt[idx] = gpd.iloc[i]['Count']/total

    return prob_cnjt

In [56]:
data_rain = pd.read_table("teste_rain.cas", index_col=0)

In [315]:
a = joint_probability_new([1,1],data_rain, ['Sprinkler', 'Rain'])

[[False, False], [False, True], [True, False], [True, True]]


,Sprinkler,Rain,Count
0,False,False,4820
1,False,True,2002
2,True,False,3161
3,True,True,17


[[False, False], [False, True], [True, False], [True, True]]

In [168]:
a

array([0.482 , 0.2002, 0.3161, 0.0017])

In [148]:
combinations = list(map(list, itertools.product([False, True, 'Term'], repeat=np.sum(2))))

In [257]:
def cond_prob(prob_conj_xy, x_bits, y_bits):
    '''Calculates the probability of x | y'''
    prob_cond = np.zeros((x_bits, y_bits), dtype=float)
    
    for i in np.arange(0, y_bits):
        y_tot = np.sum(prob_conj_xy.T[i])
                    
        for j, element in enumerate(prob_conj_xy.T[i]):
            print('conj ',prob_conj_xy[j], '\n')
            print(element)
            print(y_tot)
            if y_tot != 0:
                prob_cond[i][j] =element/y_tot
                print(prob_cond)
            
    return prob_cond


In [183]:
conj = [0.4, 0.2,0.1,0.3]

In [258]:
cond_prob(b, 2,2)

conj  [0.482  0.2002] 

0.482
0.7981
[[0.60393434 0.        ]
 [0.         0.        ]]
conj  [0.3161 0.0017] 

0.3161
0.7981
[[0.60393434 0.39606566]
 [0.         0.        ]]
conj  [0.482  0.2002] 

0.2002
0.2019
[[0.60393434 0.39606566]
 [0.99157999 0.        ]]
conj  [0.3161 0.0017] 

0.0017
0.2019
[[0.60393434 0.39606566]
 [0.99157999 0.00842001]]


array([[0.60393434, 0.39606566],
       [0.99157999, 0.00842001]])

In [246]:
b = a.reshape(2,2).copy()

In [247]:
b

array([[0.482 , 0.2002],
       [0.3161, 0.0017]])

In [319]:
c = joint_probability_new([1,1,1],data_rain, ['GRASSWET','Sprinkler', 'Rain', ])

[[False, False, False], [False, False, True], [False, True, False], [False, True, True], [True, False, False], [True, False, True], [True, True, False], [True, True, True]]


,GRASSWET,Sprinkler,Rain,Count
0,False,False,False,4820
1,False,False,True,193
2,False,True,False,611
3,True,False,True,1809
4,True,True,False,2550
5,True,True,True,17


[[False, False, False],
 [False, False, True],
 [False, True, False],
 [False, True, True],
 [True, False, False],
 [True, False, True],
 [True, True, False],
 [True, True, True]]

In [322]:
c

array([0.482 , 0.0193, 0.0611, 0.    , 0.    , 0.1809, 0.255 , 0.0017])

In [300]:
def conditional_prob(k,l,joint_prob):
    states = list(map(list, itertools.product([False, True], repeat=k+l)))
    combinations = list(map(list, itertools.product([False, True], repeat=k+l+1)))

    size = int(joint_prob.size/2)
    conditional = np.zeros(2**(k+l+1))

    for i,state in enumerate(states):
        index_zero = combinations.index([False] + state)
        print(([False] + state))
        prob_zero = joint_prob[index_zero]

        index_one = combinations.index([True] + state)
        print(([True] + state))
        prob_one = joint_prob[index_one]

        if(prob_zero + prob_one != 0):
            conditional[i] = prob_zero/(prob_zero+ prob_one)
            conditional[i + 2**(k+l)] = prob_one/(prob_zero+ prob_one)
    return conditional

In [326]:
conditional_prob(1,1,c)

[False, False, False]
[True, False, False]
[False, False, True]
[True, False, True]
[False, True, False]
[True, True, False]
[False, True, True]
[True, True, True]


array([1.        , 0.0964036 , 0.19329326, 0.        , 0.        ,
       0.9035964 , 0.80670674, 1.        ])

In [302]:
[[False, False, False], [False, False, True], [False, True, False], [False, True, True], [True, False, False], [True, False, True], [True, True, False], [True, True, True]]

[[False, False, False],
 [False, False, True],
 [False, True, False],
 [False, True, True],
 [True, False, False],
 [True, False, True],
 [True, True, False],
 [True, True, True]]